###

# Deep Learning (RNN) Demo for Load Forecasting

### Step 1: Import all the packages needed

In [1]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import random as rd
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl

### Step 2: setting all global parameters -- sec 2 network configuration

In [2]:
time1 = time.time() # set up counter to record run time
data_dir = './data/' # directory contains input data
num_epoches = 6000 # training epoches for each customer samples
n_steps = 48 # input size
test_batch_size = 70*48 # days of a batch
train_batch_size = 10*48
feature_size = 1 # same time of a week
n_hidden = 20 # input size
num_layers = 3
n_output = 1
Rs = 50

### Step 4: define data generating function code. 
which generate a batch of batch-size large sequence data. the data is feature_size dims width and is a time series of float32 of steps steps. inputs and outputs are:

inputs:
----n_batch: number of samples in a batch
----steps: the sequence length of a sample data
----feature_size: dimensions of a single time step data frame

outputs:
----X inputs, shape(n_batch,steps,feature_size)
----Y outputs should be, shape(n_batch,)

In [3]:
def train_data_gen(totaltraindays,x_data,y_data,steps = 48, n_batch = train_batch_size):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    rang = range(totaltraindays) # test day sample range
    train_days_list = rd.sample(rang,n_batch) # pick unduplicated n indexes as examples
    #print totaltraindays
    tmpX = [x_data[i,0-steps:] for i in train_days_list]
    tmpY = [y_data[i,:] for i in train_days_list]
    X = np.array(tmpX).reshape(n_batch,steps,feature_size)
    Y = np.array(tmpY).reshape(n_batch,feature_size)
    return (X,Y)

In [4]:
def test_data_gen(x_data,y_data,steps = 48, n_batch = test_batch_size):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    #print x_data[:,0-steps:].shape,y_data.shape
    #print n_batch, steps
    X = x_data[:,0-steps:].reshape(n_batch,steps,feature_size)
    Y = y_data.reshape(n_batch,feature_size)
    return (X,Y)

### Step 5: construct RNN model

In [5]:
# create placeholder for x and y
x = tf.placeholder("float",[None,n_steps,feature_size])
istate = tf.placeholder("float",[None,num_layers*2*n_hidden])
y = tf.placeholder("float",[None,n_output])


# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([feature_size, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [6]:
def RNN(_X, _istate, _weights, _biases):

    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, feature_size]) # (n_steps*batch_size, n_input)
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    stacked_lstm_cell = rnn_cell.MultiRNNCell([lstm_cell]*num_layers)
    
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(stacked_lstm_cell, _X, initial_state=_istate)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [7]:
pred = RNN(x, istate, weights, biases)

#cost function 
cost = tf.reduce_mean(tf.pow(pred-y,2)) # cost function of this batch of data
#compute parameter updates
#optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(cost)
#optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
optimizer = tf.train.AdamOptimizer().minimize(cost)







#optimizer2 = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost2)

In [8]:
## iterating among all customers to find current training customer
accuracy = []
result_final = []
for i in range(1):
    test_x_name = data_dir + 'test_x_' + str(i) + '.csv'
    test_y_name = data_dir + 'test_y_' + str(i) + '.csv'
    train_x_name = data_dir + 'train_x_' + str(i) + '.csv'
    train_y_name = data_dir + 'train_y_' + str(i) + '.csv'
    tmp_data = np.array(pd.read_csv(test_x_name,header = None))
    test_x_data = tmp_data[:,1:]
    # print test_x_data.dtype  data are stored as float64 double precision format
    tmp_data = np.array(pd.read_csv(test_y_name,header = None))
    test_y_data = tmp_data[:,1:]
    tmp_data = np.array(pd.read_csv(train_x_name,header = None))
    train_x_data = tmp_data[:,1:]
    tmp_data = np.array(pd.read_csv(train_y_name,header = None))
    train_y_data = tmp_data[:,1:]
    #log them
    #test_x_data = np.log(test_x_data)
    #test_y_data = np.log(test_y_data)
    #train_x_data = np.log(train_x_data)
    #train_y_data = np.log(train_y_data)
    
    traindays = train_y_data.shape[0]
    # generate test data
    test_x,test_y = test_data_gen(test_x_data,test_y_data,n_steps)
    test_x = test_x.reshape(test_batch_size,n_steps,feature_size)
    ### Execute
    # Initializing the variables
    init = tf.initialize_all_variables()
    outp = []
    outlist = np.zeros([Rs,test_batch_size])
    with tf.Session() as sess:
        # Create a summary to monitor cost function
        tf.scalar_summary("loss", cost)
        # Merge all summaries to a single operator
        merged_summary_op = tf.merge_all_summaries()

        # tensorboard info.# Set logs writer into folder /tmp/tensorflow_logs
        summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def=sess.graph_def)

        #initialize all variables in the model
        sess.run(init)
        for k in range(num_epoches):
            #Generate Data for each epoch
            #What this does is it creates a list of of elements of length seq_len, each of size [batch_size,input_size]
            #this is required to feed data into rnn.rnn
            #print traindays
            X,Y = train_data_gen(traindays,train_x_data,train_y_data,n_steps)
            X = X.reshape(train_batch_size,n_steps,feature_size)


            #Create the dictionary of inputs to feed into sess.run
            #if k < 0:
            #    sess.run(optimizer2,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})
            #else:
            sess.run(optimizer,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})   
            #perform an update on the parameters
            
            if k >= num_epoches-Rs:
                output_tmp = sess.run(pred,feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
                outp_test = output_tmp
                outlist[k-num_epoches+Rs,:] = outp_test.copy().T

            # Write logs at every iteration
            if k>50 & k%10 == 0:
                summary_str = sess.run(merged_summary_op, feed_dict={x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
                summary_writer.add_summary(summary_str, k)
            
            if k % 10 == 0:
                output_tmp_ex = sess.run(pred,feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
                print "Iter " + str(k)# + ", Minibatch Loss ---- Train = " + "{:.6f}".format(loss1) + "; Test = " + "{:.6f}".format(loss2)
        #print "haha{}".format(outp)
                ktmp = np.corrcoef(output_tmp_ex.T,test_y.T)[0,1]
                accuracy.append(ktmp)
                print ktmp
    R = []
    RR  = []
    for i in range(Rs):
        out = np.array(outlist[i])
        #R.append(np.corrcoef(np.exp(out.T),np.exp(test_y.T))[0,1])
        R.append(np.corrcoef(out.T,test_y.T)[0,1])
        RR.append(np.corrcoef(out.T,test_y.T)[0,1]**2)
    print R
    RRR = np.mean(R)# average Rs R in this time of train
    
    
    # run time
    time2 = time.time()
    print 'total running time cost:{}s'.format(time2-time1)
    
    # append R
    result_final.append(RRR)

Iter 0
-0.138687726901
Iter 10
-0.188801437343
Iter 20
0.101280484665
Iter 30
0.307724778668
Iter 40
0.34969742069
Iter 50
0.372270283434
Iter 60
0.398813142633
Iter 70
0.427194887349
Iter 80
0.464317910429
Iter 90
0.483649893331
Iter 100
0.492447626467
Iter 110
0.517304714437
Iter 120
0.51851458213
Iter 130
0.535000436943
Iter 140
0.540230872684
Iter 150
0.553548726132
Iter 160
0.55862332999
Iter 170
0.569806541069
Iter 180
0.5797235111
Iter 190
0.590271695185
Iter 200
0.596979721647
Iter 210
0.608585321729
Iter 220
0.616817147084
Iter 230
0.623433112179
Iter 240
0.629480070188
Iter 250
0.635668102312
Iter 260
0.642846785161
Iter 270
0.648411612958
Iter 280
0.649625317574
Iter 290
0.655603001333
Iter 300
0.661715288225
Iter 310
0.665712094858
Iter 320
0.670035076581
Iter 330
0.674431055756
Iter 340
0.67710725215
Iter 350
0.680937484053
Iter 360
0.680534718534
Iter 370
0.683153581878
Iter 380
0.68160777482
Iter 390
0.685113629297
Iter 400
0.685869658
Iter 410
0.686493742048
Iter 420
0.

In [9]:
DataFrame(accuracy).to_csv('R-house0-size-3-20-adam.csv')
result_final

[0.71024079595938017]

In [10]:
# Draw
#xxx = np.arange(0,test_batch_size)
#pl.plot(xxx,out,color = "red")
#pl.plot(xxx,test_y)
#pl.grid()
##pl.legend()
#pl.show()

In [11]:
#out = np.array(outlist[0])
#np.corrcoef(out.T,test_y.T)

In [12]:
#DataFrame(result_final).to_csv('result2.csv')

In [13]:
#train_y_data.shape

In [14]:
#a = [1,2,3,4]

In [15]:
#a = np.array(a).reshape((2,2))

In [16]:
#a

In [17]:
#b = np.log(a)

In [18]:
##b

In [19]:
#c = np.exp(b)

In [20]:
#c

In [21]:
help(tf.train.AdamOptimizer)

Help on class AdamOptimizer in module tensorflow.python.training.adam:

class AdamOptimizer(tensorflow.python.training.optimizer.Optimizer)
 |  Optimizer that implements the Adam algorithm.
 |  
 |  See [Kingma et. al., 2014](http://arxiv.org/abs/1412.6980)
 |  ([pdf](http://arxiv.org/pdf/1412.6980.pdf)).
 |  
 |  @@__init__
 |  
 |  Method resolution order:
 |      AdamOptimizer
 |      tensorflow.python.training.optimizer.Optimizer
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
 |      Construct a new Adam optimizer.
 |      
 |      Initialization:
 |      
 |      ```
 |      m_0 <- 0 (Initialize initial 1st moment vector)
 |      v_0 <- 0 (Initialize initial 2nd moment vector)
 |      t <- 0 (Initialize timestep)
 |      ```
 |      
 |      The update rule for `variable` with gradient `g` uses an optimization
 |      described at the end of section2 of th